Step 1

In [1]:
#Before we start Here are the things we have to create
    #Create subscription
    #Create storage acc -> create container where we will save our files (we can use data factory, sql, cosmo all possible)
    #Create resource group - machine learning west us-2
    #Create container registry - create
    #Create compute engine - create cpu (if you have already created somewhere you can attach it to here)
    #Create create datastore - get the access key from storageaccount->Accesss Key->Key

    #Note : If we upload something on AML notebook home, it get save to defaultblobstorage , to see navigate to datastore

    #in config.py below
    #Workspace details -> Go to\ AML home, download json file

In [19]:
import seaborn as sns

<!-- -> Created Resource Group
-> Created AML Workspace
-> Blob Storage Account + Create Container
-> Compute Engine
-> Data Store in Data Section -->

Step 2

In [ ]:
#Once done we will create 3 files, one for each step in ml
#for my case read_data, prep and modeling

Step 3

In [11]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, date, timedelta
import argparse
import azureml.core
from azureml.core import Workspace, Dataset, Datastore
from azureml.core.compute import ComputeTarget
from azureml.core.runconfig import RunConfiguration 
from azureml.core.conda_dependencies import CondaDependencies 
from azureml.core import Environment 
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline
from azureml.core import Experiment
from azureml.core.authentication import InteractiveLoginAuthentication

Workspace

In [12]:
ws = Workspace.from_config() 

In [13]:
compute_name = "ram"
vm_size = "STANDARD_DS11_V2"
compute_target = ws.compute_targets[compute_name]

Environment Setup

In [31]:
#Declaring environment
aml_config = RunConfiguration()
aml_config.target = compute_target 

USE_CURATEDUENV = True 
if USE_CURATEDUENV:
    curated_env = Environment.get(workspace = ws, name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")
    aml_config.environment = curated_env
else:
    aml_config.environment.python.user_managed_dependencies = False
    #inside this env, setting up what all libraries will be needed
    aml_config.environment.python.conda_dependencies = CondaDependencies.create(
    #Package that will be required during the prep step
    conda_packages=['pandas','scikit-learn'],
    pip_packages=['azureml-sdk', 'azureml-dataset-runtime[fuse, pandas]', 'seaborn'],
    pin_sdk_version = False)

Pipeline

In [117]:
#Pipeline
read_data = 'data_wrangling.py'
prep = 'preprocessing.py'
model = 'modeling.py'

#Script initialization
py_script_run_read = PythonScriptStep(
                script_name=read_data,
                compute_target=compute_target,
                arguments=['--input-data','diabetes.csv'],
                runconfig = aml_config,
                allow_reuse=False)

py_script_run_prep = PythonScriptStep(
                script_name=prep,
                compute_target=compute_target,
                arguments=['--prep','wranggled.csv'],
                runconfig = aml_config,
                allow_reuse=False)

py_script_run_model = PythonScriptStep(
                script_name=model,
                compute_target=compute_target,
                arguments=['--train','preprocessed.csv'],
                runconfig = aml_config,
                allow_reuse=False)

pipeline_step = [py_script_run_read, py_script_run_prep, py_script_run_model]
pipeline_1 = Pipeline(workspace=ws, steps=[pipeline_step])

Experiment

In [118]:
pipeline_run = Experiment(ws, "First_run").submit(pipeline_1)
pipeline_run.wait_for_completion(show_output=True)

Created step data_wrangling.py [952a9f0a][dc197810-a784-4ec6-857e-dfac464b1c46], (This step will run and generate new outputs)
Created step preprocessing.py [70e27e53][0c12deca-bd8c-4dac-85f3-6e2ea0206bbb], (This step will run and generate new outputs)
Created step modeling.py [08a5a31b][ae864442-9fbd-4248-be9a-e5299c5943bc], (This step will run and generate new outputs)
Submitted PipelineRun d0d35c90-5744-4342-b96d-d59c1fc7dcfe
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/d0d35c90-5744-4342-b96d-d59c1fc7dcfe?wsid=/subscriptions/82cfc0a9-b871-4f48-bb9a-0f9e00faacf9/resourcegroups/ram/workspaces/ram&tid=48dbabfc-37fa-4dd4-913c-cd7091a8ea08
PipelineRunId: d0d35c90-5744-4342-b96d-d59c1fc7dcfe
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/d0d35c90-5744-4342-b96d-d59c1fc7dcfe?wsid=/subscriptions/82cfc0a9-b871-4f48-bb9a-0f9e00faacf9/resourcegroups/ram/workspaces/ram&tid=48dbabfc-37fa-4dd4-913c-cd7091a8ea08
PipelineRun Status: NotStarted
PipelineRun Stat

'Finished'

Select & Register Best Model

In [119]:
pipeline_run

Experiment,Id,Type,Status,Details Page,Docs Page
First_run,d0d35c90-5744-4342-b96d-d59c1fc7dcfe,azureml.PipelineRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [120]:
datastore = Datastore.get(ws, 'workspaceblobstore')
datastore.download(target_path="azureml", prefix="model_estimator", overwrite=False, show_progress=True)


Path already exists. Skipping download for azureml/model_estimator_500.pkl


0

In [121]:
# #run_context = pipeline_run.get_context()
# bring the model to cwd
from azureml.core.model import Model
finetuning_model = Model.register(model_name='model_estimator_500',
                                              model_path="./azureml/model_estimator_500.pkl",
                                              tags={},
                                              description="Diabetes Model", workspace=ws)

Registering model model_estimator_500


In [122]:
# Load the model
from azureml.core.model import Model
model = Model(ws, name='model_estimator_500')
print("Loaded model version:", model.version)

Loaded model version: 2


Deployment

Builing image using
 - We need score.py
 - model file
 - myenv.yml the environment file

In [123]:
import sklearn

In [124]:
from azureml.core import Environment

env = Environment("deploytocloudenv_2comp")
env.python.conda_dependencies.add_pip_package("joblib")
env.python.conda_dependencies.add_pip_package("numpy==1.23")
env.python.conda_dependencies.add_pip_package("scikit-learn=={}".format(sklearn.__version__))

In [98]:
%%writefile score3.py
import joblib
import json
import numpy as np

from azureml.core.model import Model

def init():
    global model_3
    model_3_path = Model.get_model_path(model_name='model_estimator_500')
    model_3 = joblib.load(model_3_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = np.array(data)
        result_1 = model_3.predict(data)
        
        return {"prediction1": result_1.tolist()}
    except Exception as e:
        result = str(e)
        return result

Writing score3.py


In [99]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score3.py", environment=env)

In [100]:
from azureml.core.webservice import AciWebservice

aci_service_name = "aciservice-modeldiabetes"

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(ws, aci_service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)

print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-02-03 14:21:53+00:00 Creating Container Registry if not exists.
2023-02-03 14:21:56+00:00 Use the existing image.
2023-02-03 14:21:56+00:00 Generating deployment configuration.
2023-02-03 14:21:57+00:00 Submitting deployment to compute.
2023-02-03 14:22:05+00:00 Checking the status of deployment aciservice-modeldiabetes..
2023-02-03 14:26:08+00:00 Checking the status of inference endpoint aciservice-modeldiabetes.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [115]:
datastore = Datastore.get(ws, 'workspaceblobstore')
df = Dataset.Tabular.from_delimited_files(path=[(datastore, "preprocessed.csv")]).to_pandas_dataframe()
df.head()

df = df[['Pregnancies', 'Glucose', 'SkinThickness', 'BMI', 'Age',
       'Outcome']]

df_new = np.array(df)

In [116]:
import json
test_sample = json.dumps({'data': df_new[:4].tolist()})
predictions = service.run(test_sample)
predictions

{'prediction1': [1.0, 0.0, 0.0, 0.0]}